First we split the dataset. This only needs to be done once. We use a seed to ensure consistent results.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import timm
from pathlib import Path

from utils.pytorch_models import ResNet18
from models.poolformer import create_poolformer_s12
from models.ConvMixer import create_convmixer_1024_20
from models.MLPMixer import create_mlp_mixer
from utils.clients import GlobalClient
from utils.pytorch_utils import start_cuda

In [ ]:
c1_path = Path("data/c1_train.csv")
c2_path = Path("data/c2_train.csv")
c3_path = Path("data/c3_train.csv")
if not (c1_path.exists() and c2_path.exists() and c3_path.exists()):
    df = pd.read_csv("data/train.csv", header=None)

    seed = 42

    df_c1, temp = train_test_split(df, test_size=0.7, random_state=seed)
    df_c2, df_c3 = train_test_split(df, test_size=0.5, random_state=seed)

    df_c1.to_csv("data/c1_train.csv", index=False, header=None)
    df_c2.to_csv("data/c2_train.csv", index=False, header=None)
    df_c3.to_csv("data/c3_train.csv", index=False, header=None)

In [ ]:
cuda_no = 1
batch_size = 128
num_workers = 0
epochs = 1
communication_rounds = 40

channels = 10
num_classes = 19
dataset_filter = "serbia"

## Train ConvMixer

In [ ]:
convmixer = create_convmixer_1024_20(channels, num_classes)
global_client_convmixer = GlobalClient(
    model=convmixer,
    lmdb_path="data/BigEarth_Serbia_Summer_S2.lmdb",
    val_path="data/test.csv",
    csv_paths=["data/c1_train.csv", "data/c2_train.csv", "data/c3_train.csv"],
)

In [ ]:
import json
global_convmixer_results = global_client_convmixer.train(communication_rounds=communication_rounds, epochs=epochs)


with open('results/convmixer_results.json', 'w') as f:
    json.dump(global_convmixer_results, f)

## Train PoolFormer

In [ ]:
poolformer_s12 = create_poolformer_s12(in_chans=channels, num_classes=num_classes)
global_client_poolformer = GlobalClient(
    model=poolformer_s12,
    lmdb_path="data/BigEarth_Serbia_Summer_S2.lmdb",
    val_path="data/test.csv",
    csv_paths=["data/c1_train.csv", "data/c2_train.csv", "data/c3_train.csv"],
)

In [ ]:
import json
global_poolformer_results = global_client_poolformer.train(communication_rounds=communication_rounds, epochs=epochs)


with open('results/poolformer_results.json', 'w') as f:
    json.dump(global_poolformer_results, f)

## Train ResNet18

In [ ]:
resnet18 = ResNet18(num_cls=num_classes, channels=channels, pretrained=True)
global_client_resnet18 = GlobalClient(
    model=resnet18,
    lmdb_path="data/BigEarth_Serbia_Summer_S2.lmdb",
    val_path="data/test.csv",
    csv_paths=["data/c1_train.csv", "data/c2_train.csv", "data/c3_train.csv"],
)

In [ ]:
import json
global_resnet18_results = global_client_resnet18.train(communication_rounds=communication_rounds, epochs=epochs)


with open('results/resnet18_results.json', 'w') as f:
    json.dump(global_resnet18_results, f)

## Train MLP-Mixer

In [ ]:
mlp_mixer = create_mlp_mixer(channels, num_classes)
global_client_mlp_mixer = GlobalClient(
    model=mlp_mixer,
    lmdb_path="data/BigEarth_Serbia_Summer_S2.lmdb",
    val_path="data/test.csv",
    csv_paths=["data/c1_train.csv", "data/c2_train.csv", "data/c3_train.csv"],
)

In [ ]:
import json
global_mlp_mixer_results = global_client_mlp_mixer.train(communication_rounds=communication_rounds, epochs=epochs)


with open('results/mlp_mixer_results.json', 'w') as f:
    json.dump(global_mlp_mixer_results, f)